# Mock Mosaiq DB for testing session offset logic
This notebook uses the [pandas.DataFrame.to_sql](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html) function to create a mock database with slimmed-down Site, Offset, and Dose_Hst tables.  These DataFrames are populated from randomly generated data.

## Define protocols and treatment techniques
For each mock tx sites, randomly chosen:
* number of fractions
* treatment techniques, which determines the number of fields
* trending/imaging protocols, encoded in the Site Notes column

There are three trending/imaging protocols defined:
* weekly verification imaging -- for example imaging every Friday.  assumes that a session offset is created to record the magnitude of the shift
* daily imaging -- image daily and compute a session offset
* NAL protocol -- image daily and compute session offset for the first 4 sessions, then weekly verification.  

Each protocol is characterized by the probability of a session offset being generated for each session.  This probability changes for the NAL protocol, so a function of session number is defined for each protocol.

See Lozano, Eva M., et al. "Correction of systematic set-up error in breast and head and neck irradiation through a no-action level (NAL) protocol." Clinical and Translational Oncology 13.1 (2011): 34-42.

In [ ]:
# vary the number of fractions a bit
number_of_fractions = [20, 25, 30]

# dict that represents the number of fields for a few named techniques
field_count_by_technique_names = {
    '4-fld': 4, 
    '5-fld': 5, 
    '7-fld': 7 
}

# dict of functions that return the percent of sessions 
# with an offset, as a function of session number
prob_offset_by_protocol = {
    'weekly': lambda _: 20,  # always 20% ~ once per week
    'daily': lambda _: 90,   # always 90% ~ daily, with occasional misses
    'nal': lambda session_num: 95 if session_num <= 4 else 20
}

Given these definitions, generate a list of sites by randomly choosing the protocol, treatment technique, and number of fractions.

Encode the sites on a pandas DataFrame, and then add the columns to support the Mosaiq versioned entity for sites (i.e. SIT_SET_ID and Version).

In [ ]:
import pandas as pd
from random import randint, choices
from datetime import datetime, timedelta
from decimal import Decimal

site_count = 10   # how many site records to create?

# the site notes contain the choice of protocol
site_notes = choices(list(prob_offset_by_protocol.keys()),
                     weights=[1,1,1], k=site_count)

# the treatment technique is chosen from the list of keys
site_techniques = choices(list(field_count_by_technique_names.keys()), 
                          weights=[2,1,3], k=site_count)

# choose the number of fractions
site_fractions = choices(number_of_fractions, 
                        weights=[1,2,3], k=site_count)

# now create the dataframe
site_df = pd.DataFrame(zip(site_notes, site_techniques, site_fractions),
                        columns=['Notes', 'Technique', 'Fractions'])
site_df.index += 1   # index = SIT_ID, so start from 1
site_df['Pat_ID1'] = site_df.index + 10000   # use the SIT_ID+10000 as the Pat_ID1
site_df['SIT_SET_ID'] = site_df.index
site_df['Version'] = 0
site_df['Site_Name'] = 'rx1'
display(site_df)

## Generate treatment records for each site
For each site we pick a time to begin the treatment sessions, and then start generating sessions (Dose_Hst and Offset) based on the site parameters.

The sessions are formed based on both a starting date and an appointment time.
* Treatments occur at roughly the same time each workday
* Offsets happen at a frequency determined by the specified protocol
* Offset shift values are randomly generated
* Dose_Hst records are created based on the number of fractions, with randomly generated FLD_IDs

In [ ]:
import re
from decimal import Decimal

# lists to store the offsets and dose_hst records
offset_recs, dose_hst_recs = [], []

# iterate over the rows in the site dataframe
for sit_id, site_rec in site_df.iterrows():
    # extract site description from the record
    pat_id1 = site_rec['Pat_ID1']
    sit_set_id = site_rec['SIT_SET_ID']
    fractions = site_rec['Fractions']
    protocol = re.match('([a-z]*)',site_rec['Notes']).groups()[0]
    fld_count = field_count_by_technique_names[site_rec['Technique']]
    fld_ids = [randint(1000,4000) for _ in range(fld_count)]
    
    # pick a date for beginning the treatment, as a workday number in the year
    session_workday = randint(0,200)
    # pick the appointment time between 8AM and 5pm
    appointment_time = timedelta(hours=randint(8,17))
    for n in range(fractions):
        # determine the session date for the current workday
        session_date_str = f"2021-W{session_workday//5+1}-{session_workday%5+1}"
        session_date = datetime.strptime(session_date_str, '%Y-W%W-%w')
        
        # and add the appointment time
        session_time = session_date + appointment_time
        
        # choose whether to generate an offset record
        if randint(0,100) < prob_offset_by_protocol[protocol](n):
            session_time += timedelta(minutes=randint(2,5))
            offset_recs.append((sit_set_id, session_time,
                                1, # Offset_State: 1=Active, 2=Complete
                                3, # Offset_Type: 3=Portal, 4=ThirdParty
                                Decimal(randint(-50, 50))/10, # Superior_Offset
                                Decimal(randint(-50, 50))/10, # Anterior_Offset
                                Decimal(randint(-50, 50))/10  # Lateral_Offset
                               ))
            
        # generate dose_hst by field count
        for fld_id in fld_ids:
            session_time += timedelta(minutes=randint(3,6))
            dose_hst_recs.append((pat_id1, sit_id, fld_id, session_time))

        # occasionally skip a workday
        session_workday += 1 if randint(0,5) else 2

print(f"Generated {len(offset_recs)} offsets "
      f"and {len(dose_hst_recs)} dose_hst records")

Create the Dose_Hst dataframe with appropriate column names and primary key.

In [ ]:
dose_hst_df = pd.DataFrame(dose_hst_recs, 
                           columns=['Pat_ID1', 'SIT_ID', 'FLD_ID', 'Tx_DtTm'])
dose_hst_df.index += 1  # start ID at 1
display(dose_hst_df)

Create the Offset dataframe with the versioned entity columns and the three translational offset components.

In [ ]:
offset_df = pd.DataFrame(offset_recs, 
                         columns=[
                             'SIT_SET_ID', 
                             'Study_DtTm',
                             'Offset_State',
                             'Offset_Type',
                             'Superior_Offset', 
                             'Anterior_Offset', 
                             'Lateral_Offset'])
offset_df.index += 1  # start ID at 1
offset_df['OFF_SET_ID'] = offset_df.index
offset_df['Version'] = [0] * len(offset_df)
display(offset_df)

## Populate SQL tables for Site, Offset, and Dose_Hst
Using the pandas [DataFrame.to_sql](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html) function, populate tables from the dataframes, initially in a database name *MosaiqTest94086* (to avoiding conflicting with an existing Mosaiq DB)
* Site from site_df
* Offset from offset_df
* Dose_Hst from dose_hst_df

In [ ]:
from sqlalchemy import create_engine

msq_server = '.'
test_db_name = 'MosaiqTest94086'
conn_str = f"mssql+pymssql://{msq_server}/{test_db_name}"

engine = create_engine(conn_str, echo=False)

site_df.to_sql('Site', engine, if_exists='replace', index=True, index_label='SIT_ID')
offset_df.to_sql('Offset', engine, if_exists='replace', index=True, index_label='OFF_ID')
dose_hst_df.to_sql('Dose_Hst', engine, if_exists='replace', index=True, index_label='DHS_ID')

## Query for Sites
With the test database populated, pymedphys can be used to query for sites in the Site table, selecting only the tip version.  

For each site display SIT_ID, SIT_SET_ID, Site_Name, and Notes (which should be the protocol).

In [ ]:
from pymedphys import mosaiq

with mosaiq.connect(msq_server, database=test_db_name) as cursor:
    result = mosaiq.execute(cursor,
        """
        SELECT 
            SIT_ID, 
            SIT_SET_ID, 
            Site_Name,
            Notes
        FROM Site 
        WHERE 
            Version = 0
        """)
        
    for site in result:
        sit_id, sit_set_id = site[0], site[1]
        print(f"SIT_ID:{sit_id}  SIT_SET_ID:{sit_set_id}  Site_Name:{site[2]}  Notes:{site[3]}")

Now that the database is ready, try the [demo_session_offsets](./demo_session_offsets.ipynb) notebook to look at some more complex logic.